In [5]:
import torch
import torch.nn as nn
from torch.distributions import Categorical

class DiscretePolicyNetwork(nn.Module):
   def __init__(self, state_dim, action_dim):
       super(DiscretePolicyNetwork, self).__init__()
       self.fc1 = nn.Linear(state_dim, 128)
       self.fc2 = nn.Linear(128, action_dim) # Выходной слой = количеству действий
       self.activation = nn.ReLU()

   def forward(self, x): # Вход - тензор состояния
       x = self.fc1(x)
       x = self.activation(x)
       x = self.fc2(x)
       return x # Выход - логиты (до softmax)

   def act(self, state):
       """
       Выбирает действие для дискретного пространства
       и возвращает его вместе с log_prob.
       state: numpy array или torch.Tensor
       """
       state_tensor = torch.from_numpy(state).float().unsqueeze(0) # Добавляем размерность батча
       logits = self(state_tensor) # Пропускаем состояние через сеть
       dist = Categorical(logits=logits) # Создаем категориальное распределение из логитов
       action = dist.sample() # Сэмплируем действие из распределения
       log_prob = dist.log_prob(action) # Получаем log_prob для выбранного действия
       return action.item(), log_prob

# Пример использования:
policy_discrete = DiscretePolicyNetwork(state_dim=4, action_dim=2)
dummy_state = torch.randn(4).numpy() # Имитация состояния
action, log_prob = policy_discrete.act(dummy_state)
print(f"Дискретное действие: {action}, Log-вероятность: {log_prob.item()}")

Дискретное действие: 1, Log-вероятность: -0.6342518329620361


In [6]:
import torch
import torch.nn as nn
from torch.distributions import Normal

class ContinuousPolicyNetwork(nn.Module):
   def __init__(self, state_dim, action_dim):
       super(ContinuousPolicyNetwork, self).__init__()
       self.fc1 = nn.Linear(state_dim, 128)
       self.fc_mu = nn.Linear(128, action_dim) # Выход для среднего значения (mu)
       self.fc_log_std = nn.Linear(128, action_dim) # Выход для логарифма стандартного отклонения (log_std)
       self.activation = nn.ReLU()
       self.tanh = nn.Tanh()

   def forward(self, x):
       # 1. Пропускаем состояние через сеть
       x = self.fc1(x)
       x = self.activation(x)
       # 2. Используем полученное скрытое состояние x
       # для вычисления mu и std
       # 2a. Вычисляем mu
       mu = self.tanh(self.fc_mu(x))  # tanh ограничит mu в диапазоне [-1, 1]
       # 2b. Вычисляем std
       log_std = self.fc_log_std(x)  # сначала log_std
       std = torch.exp(log_std)  # затем exp, чтобы std получилось > 0
       return mu, std

   def act(self, state):
       """
       Выбирает действие для непрерывного пространства и возвращает его вместе с log_prob.
       state: numpy array или torch.Tensor
       """
       state_tensor = torch.from_numpy(state).float().unsqueeze(0) # Добавляем размерность батча
       mu, std = self(state_tensor) # Пропускаем состояние через сеть
       dist = Normal(mu, std) # Создаем нормальное распределение
       action = dist.sample() # Сэмплируем действие
       log_prob = dist.log_prob(action).sum(axis=-1) # Суммируем log_prob по всем измерениям действия
       return action.squeeze(0).cpu().numpy(), log_prob.item()

# Пример использования:
policy_continuous = ContinuousPolicyNetwork(state_dim=4, action_dim=1) # Например, для CartPole с непрерывным управлением
dummy_state = torch.randn(4).numpy()
action, log_prob = policy_continuous.act(dummy_state)
print(f"Непрерывное действие: {action}, Log-вероятность: {log_prob}")

Непрерывное действие: [0.1556711], Log-вероятность: -1.072463035583496
